In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble, Aer

In [2]:
dev = qml.device('qiskit.aer', wires = 4)

def layer(weights):
    # Entanglement block
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    # u3 gate
    qml.Rot(weights[0, 0], weights[0, 1], weights[0, 2], wires=0)
    qml.Rot(weights[1, 0], weights[1, 1], weights[1, 2], wires=1)
    qml.Rot(weights[2, 0], weights[2, 1], weights[2, 2], wires=2)
    qml.Rot(weights[3, 0], weights[3, 1], weights[3, 2], wires=3)

In [3]:
def encoder(encodings):
        return [i for i, b in enumerate(encodings) if b == '1']

@qml.qnode(dev, interface='torch')
def qc(weights, encoding = None):
    # encoding
    if encoding:
        encoder = encoder(encoding)
        qml.RX(np.pi, wires=encoder)
        qml.RZ(np.pi, wires=encoder)
    #layerwise
    for w in weights:
        layer(w)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]
    
def variational_qc(weights, bias, encoding = None):
    return qc(weights, encoding = encoding) + bias


In [4]:
def mse(targs, preds):
    return sum([(tar - pred)**2 for tar, pred in zip(targs, preds)])/len(targs)

def cost(weights, bias, batch_features, v_targets):
    v_preds = [variational_qc(weights, bias, encoding=b['state'])[b['action']] for b in batch_features]
    return mse(v_targets, v_preds)